In [1]:
# Import Dependencies
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import csv
from sqlalchemy import create_engine
import codecs
import re
import ast


requests.__version__

'2.27.1'

In [2]:
# Import the API key

from config import yelp_api_key


In [3]:
city_file_to_load = "Resources/zip_code_database.csv"
# Open the city_file to filter zip codes for metropolitan areas.
city_file_df = pd.read_csv(city_file_to_load)

In [4]:
cities_df = city_file_df[(city_file_df.zip >= 10001) & 
                         (city_file_df.irs_estimated_population >= 50000) &
                        (city_file_df.state.isin(["WA", "IL", "NY"]))]

cities_df.head(10)

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population
3762,10002,STANDARD,0,New York,Knickerbocker,"Manhattan, New York City, NY, Ny City, Nyc",NY,New York County,America/New_York,718,NaN,US,40.71,-73.99,63960
3783,10023,STANDARD,0,New York,NaN,"Manhattan, New York City, NY, Ny City, Nyc",NY,New York County,America/New_York,"212,347,646,718,917",NaN,US,40.78,-73.98,50320
3785,10025,STANDARD,0,New York,NaN,"Manhattan, New York City, NY, Ny City, Nyc",NY,New York County,America/New_York,"212,646,917",NaN,US,40.80,-73.97,67440
3789,10029,STANDARD,0,New York,NaN,"Manhattan, New York City, NY, Ny City, Nyc",NY,New York County,America/New_York,"212,646,917",NaN,US,40.79,-73.94,57070
3931,10306,STANDARD,0,Staten Island,NaN,NaN,NY,Richmond County,America/New_York,718,NaN,US,40.58,-74.14,52070
3937,10312,STANDARD,0,Staten Island,NaN,NaN,NY,Richmond County,America/New_York,"347,917,929,718",NaN,US,40.55,-74.18,56690
3939,10314,STANDARD,0,Staten Island,NaN,NaN,NY,Richmond County,America/New_York,"347,718,917,929",NaN,US,40.60,-74.17,80070
3941,10452,STANDARD,0,Bronx,NaN,"Highbridge, Stadium, University Heights",NY,Bronx County,America/New_York,"347,646,917,929,212,718",NaN,US,40.84,-73.92,68040
3942,10453,STANDARD,0,Bronx,NaN,NaN,NY,Bronx County,America/New_York,718,NaN,US,40.85,-73.91,70760
3945,10456,STANDARD,0,Bronx,NaN,Morrisania,NY,Bronx County,America/New_York,"718,212,347,646,917,929",NaN,US,40.83,-73.91,77400


In [5]:
# convert cities_df to a list to loop through a list of zip codes for the api
cities = cities_df["zip"].tolist()
cities

[10002,
 10023,
 10025,
 10029,
 10306,
 10312,
 10314,
 10452,
 10453,
 10456,
 10457,
 10458,
 10462,
 10463,
 10466,
 10467,
 10468,
 10469,
 10472,
 10473,
 10701,
 10950,
 10977,
 11201,
 11203,
 11204,
 11206,
 11207,
 11208,
 11209,
 11210,
 11211,
 11212,
 11213,
 11214,
 11215,
 11218,
 11219,
 11220,
 11221,
 11223,
 11226,
 11229,
 11230,
 11233,
 11234,
 11235,
 11236,
 11354,
 11355,
 11368,
 11372,
 11373,
 11375,
 11377,
 11385,
 11432,
 11434,
 11550,
 11706,
 11717,
 11746,
 11758,
 14221,
 14580,
 60004,
 60016,
 60056,
 60073,
 60085,
 60402,
 60453,
 60505,
 60608,
 60614,
 60617,
 60618,
 60620,
 60623,
 60625,
 60628,
 60629,
 60632,
 60634,
 60638,
 60639,
 60640,
 60641,
 60647,
 60651,
 60657,
 60804,
 98012,
 98052,
 98208,
 98391,
 98682,
 99208,
 99301]

In [6]:
# count of zip codes
len(cities)

99

In [21]:
# Starting URL for Yelp API call.
# url = "https://api.yelp.com/v3/businesses/search?location="
data_file = open('data_file.json', 'w', encoding="utf-8")

#csv_writer = csv.writer(data_file)

restaurants = {"data":[]}
#
for city in cities:
    
    # headers contain the api key.
    headers = {'Authorization': 'Bearer {}'.format(yelp_api_key)}
    search_api_url = 'https://api.yelp.com/v3/businesses/search'
    params = {'term': 'Restaurants', 
              'location': city,
              'limit': 50}
    
    response = requests.get(search_api_url, headers=headers, params=params, timeout=5)
    
    data_dict = response.json()
    fifty_set = {}
    if "businesses" in data_dict:
        fifty_set[city] = []
            
        for restaurant in data_dict["businesses"]:
            #print(restaurant.values())
            #csv_writer.writerow(restaurant.values())
            fifty_set[city].append(restaurant)
            
          #  fifty_set.append(restaurant)
            
        restaurants["data"].append(fifty_set)
       
json.dump(restaurants, data_file, indent=4, sort_keys=True)
data_file.close()


In [8]:
import os
print(os.getcwd())

C:\Users\Junlee\Desktop\DataClass\Yelp_Metro_US_Analysis


In [23]:
# Create file directory variable (update directory)
file_dir = 'C:/Users/Junlee/Desktop/DataClass/Yelp_Metro_US_Analysis/'

# load json file to read
with open (f'{file_dir}data_file.json',mode = 'r', encoding="utf-8") as file:
    #yelp_raw_data = pd.read_json(file, lines=True)
    #yelp_raw_data=ast.literal_eval(json.load(file))
    #pd.DataFrame(
    #yelp_raw_data=pd.read_json(codecs.open('C:/Users/Junlee/Desktop/DataClass/Yelp_Metro_US_Analysis/data_file.json','r','utf-8'), lines=True, orient='records')
    yelp_raw_data=json.load(file)
    yelp_panda_data=pd.DataFrame(yelp_raw_data)
yelp_panda_data.head(20)  
   

,data
0,{'10002': [{'alias': 'yes-apothecary-new-york-...
1,"{'10023': [{'alias': 'the-smith-new-york-3', '..."
2,{'10025': [{'alias': 'the-ellington-new-york-4...
3,"{'10029': [{'alias': 'contento-new-york-city',..."
4,{'10306': [{'alias': 'siu-kitchen-staten-islan...
5,{'10312': [{'alias': 'siu-kitchen-staten-islan...
6,{'10314': [{'alias': 'siu-kitchen-staten-islan...
7,{'10452': [{'alias': 'suyo-gastrofusion-bronx'...
8,{'10453': [{'alias': 'antonios-trattoria-bronx...
9,{'10456': [{'alias': 'suyo-gastrofusion-bronx'...


In [26]:
len(yelp_panda_data)

99

In [ ]:
#comment